# Query Feature Extraction


In [1]:
import ir_datasets
from tqdm import tqdm

MS_MARCO_V2_QUERY_DATASETS = [
    ir_datasets.load('msmarco-document-v2/train'),
    ir_datasets.load('msmarco-document-v2/dev1'),
    ir_datasets.load('msmarco-document-v2/dev2'),
    ir_datasets.create_dataset(queries_tsv="/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/resources/2021_queries.tsv")
]

In [23]:
!python3 -m spacy download en_core_web_lg

     |████████████▍                   | 320.5 MB 145.6 MB/s eta 0:00:04

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████▋           | 532.9 MB 193.8 MB/s eta 0:00:02

     |█████████████████████           | 540.6 MB 193.8 MB/s eta 0:00:02

     |█████████████████████▏          | 548.5 MB 193.8 MB/s eta 0:00:02

...

     |███████████████████████████████ | 801.4 MB 156.2 MB/s eta 0:00:01

     |███████████████████████████████▎| 808.7 MB 156.2 MB/s eta 0:00:01

     |███████████████████████████████▌| 816.4 MB 156.2 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [2]:
import spacy
nlp = spacy.load("en_core_web_lg")

In [3]:
for query_dataset in MS_MARCO_V2_QUERY_DATASETS:
    for query in query_dataset.queries_iter():
        print(query)  
        break

GenericQuery(query_id='121352', text='define extreme')
GenericQuery(query_id='2', text=' Androgen receptor define')
GenericQuery(query_id='1048579', text='what is pcnt')
GenericQuery(query_id='787021', text='what is produced by muscle')


In [6]:
def is_5w1h_query(query):
    import re
    question_words = '|'.join(i.strip() for i in ['who', 'what', 'when', 'where', 'why', 'how'])
    regex_5w1h = re.compile('^(' + question_words + r')\s.*', flags=re.IGNORECASE)
    
    return bool(regex_5w1h.match(query.strip()))

print('is_5w1h_query("how to find my luggage?") = ' + str(is_5w1h_query("how to find my luggage?")))
print('is_5w1h_query("howto find my luggage?") = ' + str(is_5w1h_query("howto find my luggage?")))

is_5w1h_query("how to find my luggage?") = True
is_5w1h_query("howto find my luggage?") = False


In [7]:
def query_length_in_tokens(query):
    return len(query.replace('\\s+', ' ').split(' '))

print('query_length_in_tokens("how to find my luggage?") = ' + str(query_length_in_tokens("how to find my luggage?")))
print('query_length_in_tokens("howto find my luggage?") = ' + str(query_length_in_tokens("howto find my luggage?")))
print('query_length_in_tokens("h o wto find my luggage?") = ' + str(query_length_in_tokens("h o wto find my luggage?")))

query_length_in_tokens("how to find my luggage?") = 5
query_length_in_tokens("howto find my luggage?") = 4
query_length_in_tokens("h o wto find my luggage?") = 6


In [8]:
def gpe_entities(query):
    ret = 0
    for ent in nlp(query).ents:
        if ent.label_ in ['GPE']:
            ret += 1
    return ret

def org_entities(query):
    ret = 0
    for ent in nlp(query).ents:
        if ent.label_ in ['ORG']:
            ret += 1
    return ret

def person_entities(query):
    ret = 0
    for ent in nlp(query).ents:
        if ent.label_ in ['PERSON']:
            ret += 1
    return ret

print("gpe_entities('what is produced by elon musk') = " + str(gpe_entities('what is produced by elon musk')))
print("gpe_entities('what is produced by muscle in the USA?') = " + str(gpe_entities('what is produced by muscle in the USA?')))
print("gpe_entities('what is produced by muscle by IBM?') = " + str(gpe_entities('what is produced by muscle by IBM?')))


print("org_entities('what is produced by elon musk') = " + str(org_entities('what is produced by elon musk')))
print("org_entities('what is produced by muscle in the USA?') = " + str(org_entities('what is produced by muscle in the USA?')))
print("org_entities('what is produced by muscle by IBM?') = " + str(org_entities('what is produced by muscle by IBM?')))


print("person_entities('what is produced by elon musk') = " + str(person_entities('what is produced by elon musk')))
print("person_entities('what is produced by muscle in the USA?') = " + str(person_entities('what is produced by muscle in the USA?')))
print("person_entities('what is produced by muscle by IBM?') = " + str(person_entities('what is produced by muscle by IBM?')))

gpe_entities('what is produced by elon musk') = 0
gpe_entities('what is produced by muscle in the USA?') = 1
gpe_entities('what is produced by muscle by IBM?') = 0
org_entities('what is produced by elon musk') = 0
org_entities('what is produced by muscle in the USA?') = 0
org_entities('what is produced by muscle by IBM?') = 1
person_entities('what is produced by elon musk') = 1
person_entities('what is produced by muscle in the USA?') = 0
person_entities('what is produced by muscle by IBM?') = 0


In [57]:
for ent in nlp("what is produced by musk in the USA?").ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

USA 32 35 GPE


In [58]:
for ent in nlp("what was introduced by IBM?").ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

IBM 23 26 ORG


In [31]:
for ent in nlp("maranda name meaning").ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

maranda 0 7 PERSON


In [41]:
from tqdm import tqdm

df = []
for query_dataset in MS_MARCO_V2_QUERY_DATASETS:
    for query in tqdm(query_dataset.queries_iter()):
        tmp = {}
        for ent in nlp(query.text).ents:
            if ent.label_ not in tmp:
                tmp[ent.label_] = 0
            tmp[ent.label_] += 1
        df += [tmp]

322196it [34:16, 156.66it/s]
4552it [00:28, 157.40it/s]
5000it [00:31, 157.32it/s]
477it [00:03, 144.31it/s]


In [43]:
import pandas as pd

pd_df = pd.DataFrame(df)
pd_df = pd_df.fillna(0)
pd_df

,CARDINAL,ORG,DATE,FAC,GPE,NORP,PERSON,LOC,EVENT,PRODUCT,TIME,MONEY,LAW,QUANTITY,WORK_OF_ART,ORDINAL,LANGUAGE,PERCENT
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332220,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332222,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
332223,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [53]:
types = []
for k in pd_df.columns:
    types += [(k, pd_df[k].mean())]

for k,v in sorted(types, key=lambda i: i[1], reverse=True):
    print(k + ':\t\t' + str(v))

GPE:		0.11564150801414703
ORG:		0.09706975694183159
PERSON:		0.09269019489803597
DATE:		0.019595153886673187
NORP:		0.014442019715554217
LOC:		0.012368124012341034
CARDINAL:		0.011179170742719543
ORDINAL:		0.005809315975618933
PRODUCT:		0.004584242606667168
FAC:		0.0037715403717360225
QUANTITY:		0.0033350891714952215
TIME:		0.003127398600346151
EVENT:		0.0017217247347430205
WORK_OF_ART:		0.0012912935510572654
LANGUAGE:		0.0011889532696214916
LAW:		0.0010866129881857175
MONEY:		0.00046956129129355103
PERCENT:		5.418014899540974e-05


# Actual Calculation of Features


In [70]:
from tqdm import tqdm

calculated_features = []
for query_dataset in MS_MARCO_V2_QUERY_DATASETS:
    for query in tqdm(query_dataset.queries_iter()):
        calculated_features += [{
            'query_id': query.query_id,
            'query_text': query.text,
            'feature_is_5w1h_query': is_5w1h_query(query.text),
            'feature_query_length_in_tokens': query_length_in_tokens(query.text),
            'feature_gpe_entities': gpe_entities(query.text),
            'feature_org_entities': org_entities(query.text),
            'feature_person_entities': person_entities(query.text)
        }]

322196it [1:41:37, 52.84it/s]
4552it [01:26, 52.85it/s]
5000it [01:34, 52.84it/s]
477it [00:09, 48.82it/s]


In [71]:
df_calculated_features = pd.DataFrame(calculated_features)
df_calculated_features.to_json('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/query-features/ms-marco-query-features.jsonl', orient='records', lines=True)

# Comparative-Questions

In [75]:
COMPARATIVE_FILES = ['2021_queries.tsv', 'docv2_dev2_queries.tsv', 'docv2_dev_queries.tsv', 'docv2_train_queries.tsv']

COMPARATIVE_QUERIES = {}

for f in COMPARATIVE_FILES:
    df_tmp = pd.read_csv('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/query-features/comparative-queries/' + f, sep='\t')
    for _, i in tqdm(df_tmp.iterrows()):
        COMPARATIVE_QUERIES[str(i['id'])] = int(i['comparative'])

477it [00:00, 7881.11it/s]
5000it [00:00, 10289.31it/s]
4552it [00:00, 9714.34it/s] 
322196it [00:34, 9348.86it/s] 


In [81]:
df_calculated_features_2 = pd.DataFrame(calculated_features)
df_calculated_features_2['feature_query_is_comparative'] = df_calculated_features_2['query_id'].apply(lambda i: COMPARATIVE_QUERIES[str(i)])
df_calculated_features_2['feature_is_5w1h_query'] = df_calculated_features_2['feature_is_5w1h_query'].apply(lambda i: int(i))
df_calculated_features_2.to_json('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/query-features/ms-marco-query-features.jsonl', orient='records', lines=True)
df_calculated_features_2

,query_id,query_text,feature_is_5w1h_query,feature_query_length_in_tokens,feature_gpe_entities,feature_org_entities,feature_person_entities,feature_query_is_comparative
0,121352,define extreme,0,2,0,0,0,0
1,510633,tattoo fixers how much does it cost,0,7,0,0,0,0
2,674172,what is a bank transit number,1,6,0,0,0,0
3,570009,what are the four major groups of elements,1,8,0,0,0,0
4,54528,blood clots in urine after menopause,0,6,0,0,0,0
...,...,...,...,...,...,...,...,...
332220,258062,how long does it take to remove wisdom tooth,1,9,0,0,0,0
332221,783076,what is personality and intellectual developme...,1,10,0,0,0,0
332222,783322,what is pharmacy science in ireland,1,6,1,0,0,0
332223,960970,when was the first gas going to ride?,1,8,0,0,0,0


In [80]:
df_calculated_features_2['feature_query_is_comparative'].mean()

0.017424937918579277

# MS-MARCO V1 Features

In [1]:
import ir_datasets
from tqdm import tqdm

MS_MARCO_V1_QUERY_DATASETS = [
    ir_datasets.load('msmarco-document/train'),
    ir_datasets.load('msmarco-document/dev'),
    ir_datasets.load('msmarco-document/eval'),
    ir_datasets.load('msmarco-document/trec-dl-2020'),
]

In [5]:
for query_dataset in MS_MARCO_V1_QUERY_DATASETS:
    for query in query_dataset.queries_iter():
        print(query)  
        break

GenericQuery(query_id='1185869', text=')what was the immediate impact of the success of the manhattan project?')
GenericQuery(query_id='174249', text='does xpress bet charge to deposit money in your account')
GenericQuery(query_id='355339', text='how to display how.close you are to.cell.tower')
GenericQuery(query_id='1030303', text='who is aziz hashim')


In [10]:
from tqdm import tqdm

calculated_features = []
for query_dataset in MS_MARCO_V1_QUERY_DATASETS:
    for query in tqdm(query_dataset.queries_iter()):
        calculated_features += [{
            'query_id': query.query_id,
            'query_text': query.text,
            'feature_is_5w1h_query': is_5w1h_query(query.text),
            'feature_query_length_in_tokens': query_length_in_tokens(query.text),
            'feature_gpe_entities': gpe_entities(query.text),
            'feature_org_entities': org_entities(query.text),
            'feature_person_entities': person_entities(query.text)
        }]

367013it [2:06:25, 48.39it/s]
5193it [01:37, 53.42it/s]
5793it [01:48, 53.35it/s]
200it [00:03, 53.82it/s]


In [12]:
import pandas as pd
df_calculated_features = pd.DataFrame(calculated_features)
df_calculated_features['feature_is_5w1h_query'] = df_calculated_features['feature_is_5w1h_query'].apply(lambda i: int(i))
df_calculated_features

,query_id,query_text,feature_is_5w1h_query,feature_query_length_in_tokens,feature_gpe_entities,feature_org_entities,feature_person_entities
0,1185869,)what was the immediate impact of the success ...,0,12,1,0,0
1,1185868,_________ justice is designed to repair the ha...,0,24,0,1,0
2,1183785,elegxo meaning,0,2,0,1,0
3,645590,what does physical medicine do,1,5,0,0,0
4,186154,feeding rice cereal how many times per day,0,8,0,0,0
...,...,...,...,...,...,...,...
378194,985594,where is kampuchea,1,3,0,0,0
378195,99005,convert sq meter to sq inch,0,6,0,0,0
378196,997622,where is the show shameless filmed,1,6,0,0,0
378197,999466,where is velbert,1,3,0,0,1


In [13]:
df_calculated_features.to_json('/mnt/ceph/storage/data-in-progress/data-research/web-search/TREC-21/marco-v1-query-features/ms-marco-v1-query-features.jsonl', orient='records', lines=True)